In [32]:
import torch 
import torch.nn as nn
import torchtext
import numpy as np
import spacy
from io import open
import nltk

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [27]:
#hyperparamter


In [35]:
TEXT = torchtext.data.Field()

In [36]:
LABEL = torchtext.data.LabelField(tensor_type=torch.FloatTensor)

In [37]:
# dataset -IMDB

train_dataset, test_dataset = torchtext.datasets.IMDB.splits(TEXT, LABEL,root='data/data/')

TypeError: 'encoding' is an invalid keyword argument for this function

In [7]:
# validation set
train_dataset, valid_dataset = train_dataset.split()

In [8]:
# model 
class ConvNet(nn.Module):
    def __init__(self, n_classes):
        super(ConvNet,self).__init__()
        # first conv layer 
         # size [32,1,28,28]
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1), # n-2k+p+1 = 28
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)             #reudes by factor of 2
        )
        # size [32,16,14,14]
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)        
        )
        #size  [32,32,7,7]
        self.fc1 = nn.Linear(7*7*32,n_classes)
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

In [9]:
model = ConvNet(n_classes).to(device)

In [10]:
#loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [11]:
total_size = len(train_loader)
# Train the model
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader,0):
        images = images.to(device)
        labels = labels.to(device)
            #forward
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        if (i+1)%500==0:
            print("Epoch {}/{} Step {}/{} : Loss {:.4f}".format(epoch+1,num_epochs,i+1, total_size,loss))

Epoch 1/5 Step 500/938 : Loss 0.3663
Epoch 2/5 Step 500/938 : Loss 0.2969
Epoch 3/5 Step 500/938 : Loss 0.2184
Epoch 4/5 Step 500/938 : Loss 0.3447
Epoch 5/5 Step 500/938 : Loss 0.1985


In [12]:
# test the model
with torch.no_grad():
    correct = 0.0
    total = 0.0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,prediction = torch.max(outputs,1)
        total += labels.size(0)
        correct += (prediction==labels).sum()
    print("Accuracy : {:.2f}".format((correct*100.0)/total))

Accuracy : 90.00
